In [ ]:
import os
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from scipy.stats import entropy

def load_and_preprocess_image(file_path):
    img = image.load_img(file_path, target_size=(299, 299))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    return img

def calculate_is(images_folder, batch_size=32):
    # Load InceptionV3 model
    model = InceptionV3(include_top=True, weights='imagenet')

    # Load and preprocess images
    images = np.array([load_and_preprocess_image(os.path.join(images_folder, f)) for f in os.listdir(images_folder)])
    n_images = images.shape[0]

    # Predict class probabilities
    predictions = model.predict(images, batch_size=batch_size)

    # Calculate Inception Score
    p_y_given_x = predictions  # Class probabilities for each image
    p_y = np.expand_dims(np.mean(p_y_given_x, axis=0), axis=0)  # Marginal distribution of classes
    kl_divergences = np.sum(p_y_given_x * (np.log(p_y_given_x + 1e-16) - np.log(p_y + 1e-16)), axis=1)
    is_score = np.exp(np.mean(kl_divergences))

    return is_score

# Example usage
generated_images_folder = 'path_to_generated_images'
is_score = calculate_is(generated_images_folder)
print(f'Inception Score: {is_score}')
